In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [14]:
# lets code a simple feed forward neural network

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 6 input channels, 16 output channels, 5x5 square convolution kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # we also need some linear transformations; these are initialised by their in_ and out_features
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 16 output ch from second layer times the kernel size 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  
        # I don't know, where 120, 84 and 10 come from; but they stem from the picture in the tutorial
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        # => from a 2x2 window take the max value. relu = Rectified Linear unit; relu(x) = max(0,x)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


In [15]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
 0.0592  0.0502  0.1575 -0.0569 -0.0207  0.0271  0.0402 -0.0373  0.0483  0.0037
[torch.FloatTensor of size 1x10]



In [18]:
net.zero_grad()
out.backward(torch.randn(1, 10))